## Understanding Metaclasses 

A class defines the rules for an object. It defines attributes, methods, operations etc. 
But in Python a class is an object in itself, right?  
Python docs: "[As in Smalltalk, classes themselves are objects.](https://docs.python.org/3/tutorial/classes.html)"? 

So if a class is an object - What defines how it behaves ? A metaclass. 

Metaclass definition: [A metaclass in Python is a class of a class that defines how a class behaves. A class is itself an instance of a metaclass.](https://www.datacamp.com/community/tutorials/python-metaclasses)


In [26]:
class SomeClass():
    x = 10
    pass 

print(type(SomeClass()))
print("x is",SomeClass().x)
print(type(SomeClass))

<class '__main__.SomeClass'>
x is 10
<class 'type'>


As expected, the SomeClass instance is of type `SomeClass` but also the class itself is of type `type`. So what ? This actually means that the syntax we write is actually a form for us to call `type`. How else can we define a class ?

In [27]:
SomeClass = type('SomeClass',(),{"x":5}) # the 3 parameters are name, bases (inherited), attributes 

print(type(SomeClass()))
print("x is",SomeClass().x)
print(type(SomeClass))

<class '__main__.SomeClass'>
x is 5
<class 'type'>


### So how do methods work ? 

In [28]:
class Foo:
    def show(self):
        print("From Foo")
        
def some_function(self):
    print("From SomeClass")

SomeClass = type('SomeClass',(Foo,),{"x":5, "some_function":some_function})

some_instance = SomeClass()
some_instance.show()
some_instance.some_function()

From Foo
From SomeClass


Now we've seen that the `class` is just a way to call the metaclass `type`, can we create our own metaclass ? 

Our metaclass will be called `Meta` and we will crete a `__new__` function:


`__new__`
* This is called before the `__init__` method. 
* Where `__init__` initialises the class, `__new__` modifies the construction of it. As such our parameters will be the same as `type`

In [29]:
class Meta(type):
    def __new__(self, class_name, bases, attributes):
        print(attributes)
        return type(class_name, bases, attributes)

In [30]:
class Bar(metaclass=Meta):
    
    parameter = 5
    
    def __init__(self, value):
        self.value = value
        
    def show(self):
        print("From Bar")
        
bar = Bar(10)
bar.show()

{'__module__': '__main__', '__qualname__': 'Bar', 'parameter': 5, '__init__': <function Bar.__init__ at 0x7fbd27ac4830>, 'show': <function Bar.show at 0x7fbd27ac4680>}
From Bar


As can be seen, we hooked into the construction of the class and printed out the attributes. This gives us a lot of power in controlling that class. For example, we can delete all the methods when the class is created (but we will leave the __ ones). 

In [31]:
import inspect 

class Meta(type):
    def __new__(self, class_name, bases, attributes):
        attrs = {}
        for key, value in attributes.items():
            if not inspect.isfunction(value) or key.startswith("__"):
                attrs[key] = value 
                
        return type(class_name, bases, attrs)

class Bar(metaclass=Meta):
    
    parameter = 5
    
    def __init__(self, value):
        self.value = value
        
    def show(self):
        print("From Bar")
        
bar = Bar(10)
bar.show()

AttributeError: 'Bar' object has no attribute 'show'

And if we run this, as expected we will get an AttributeError `'Bar' object has no attribute 'show'`

## Now onto a useful example of metaclasses

Say you have some class that you only want to have one instance of - perhaps you would want that because it performs a computation that takes long and it never needs to be computed again in the same Python Process. We can use a `Singleton` for that. [The `singleton pattern` is a software design pattern that restricts the instantiation of a class to one "single" instance.](https://en.wikipedia.org/wiki/Singleton_pattern)

In [35]:
class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

In [40]:
class Process(metaclass=Singleton):
    def __init__(self):
        self.has_processed = False
        self.__data = None
        
    def process(self):
        if not self.has_processed:
            print("Processing...")
            self.__data = [1,2,3]
            self.has_processed = True
            
    @property
    def data(self):
        self.process()
        return self.__data

In [41]:
process1 = Process()

In [42]:
print("Data is:",process1.data)

Processing...
Data is: [1, 2, 3]


#### So, above we did run the process

In [43]:
process2 = Process()

In [44]:
print("Data is:",process1.data)

Data is: [1, 2, 3]


#### And here we did not

## Caution! 
Of course meta classes are extremely powerful and can easily make you run around in circles to find your bugs - use them with caution! 

Imagine a scenario where you write the `Process` class above and you want to inherit from it to use it elsewhere. All sub-classes will also be Singletons! - and you may not see that. 

In [47]:
class SomeSubProcess(Process):
    def __init__(self, value):
        super().__init__()
        self.x = value

In [49]:
sb1 = SomeSubProcess(1)
sb2 = SomeSubProcess(10000)
print("sb1 x", sb1.x)
print("sb2 x", sb2.x)

sb1 x 1
sb2 x 1


### If you missed the fact that `Process` was a `Singleton`, it will be very hard to debug why sb2.x is not 10000.

Thanks to [Tech With Tim](https://www.youtube.com/channel/UC4JX40jDee_tINbkjycV4Sg) for his tutorials. 